In [ ]:
import numpy as np
import pandas as pd
import random
import warnings
import os
from pandas import DataFrame
from sklearn.decomposition import PCA
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold
seed = 0
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
warnings.filterwarnings("ignore")

In [ ]:
feature_list = ["pathway", "target", "enzyme", "category"]

In [ ]:
vector_size = 572
batch_size = 256
epo_num = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
df_drug = pd.read_csv("drug_features.csv")
extraction = pd.read_csv("extraction.csv")
mechanism = extraction["mechanism"]
action = extraction["action"]
drugA = extraction["drugA"]
drugB = extraction["drugB"]

In [ ]:
def feature_vector(feature_name, df):
    def Jaccard(matrix):
        matrix = np.mat(matrix)

        numerator = matrix * matrix.T

        denominator = (
            np.ones(np.shape(matrix)) * matrix.T
            + matrix * np.ones(np.shape(matrix.T))
            - matrix * matrix.T
        )

        return numerator / denominator

    all_feature = []
    drug_list = np.array(df[feature_name]).tolist()
    # Features for each drug, for example, when feature_name is target, drug_list=["P30556|P05412","P28223|P46098|……"]
    for i in drug_list:
        for each_feature in i.split("|"):
            if each_feature not in all_feature:
                all_feature.append(each_feature)  # obtain all the features
    #print("length of all feature is", len(all_feature))
    feature_matrix = np.zeros((len(drug_list), len(all_feature)), dtype=float)
    df_feature = DataFrame(
        feature_matrix, columns=all_feature
    )  # Consrtuct feature matrices with key of dataframe
    for i in range(len(drug_list)):
        for each_feature in df[feature_name].iloc[i].split("|"):
            df_feature[each_feature].iloc[i] = 1

    df_feature = np.array(df_feature)
    sim_matrix = np.array(Jaccard(df_feature))
    
    #print(feature_name + " len is:" + str(len(sim_matrix[0])))
    return sim_matrix

In [ ]:
def prepare(df_drug, feature_list, mechanism, action, drugA, drugB):
    d_label = {}
    d_feature = {}

    # Transfrom the interaction event to number
    d_event = []
    for i in range(len(mechanism)):
        d_event.append(mechanism[i] + " " + action[i])

    count = {}
    for i in d_event:
        if i in count:
            count[i] += 1
        else:
            count[i] = 1
    event_num = len(count)
    list1 = sorted(count.items(), key=lambda x: x[1], reverse=True)
    for i in range(len(list1)):
        d_label[list1[i][0]] = i

    vector = np.zeros(
        (len(np.array(df_drug["name"]).tolist()), 0), dtype=float
    )  # vector=[]
    for i in feature_list:
        #vector = np.hstack((vector, feature_vector(i, df_drug, vector_size)))
        tempvec = feature_vector(i, df_drug)
        vector = np.hstack((vector, tempvec))
    # Transfrom the drug ID to feature vector
    for i in range(len(np.array(df_drug["name"]).tolist())):
        d_feature[np.array(df_drug["name"]).tolist()[i]] = vector[i]

    # Use the dictionary to obtain feature vector and label
    new_feature = []
    new_label = []

    for i in range(len(d_event)):
        temp = np.hstack((d_feature[drugA[i]], d_feature[drugB[i]]))
        new_feature.append(temp)
        new_label.append(d_label[d_event[i]])

    new_feature = np.array(new_feature)  # 323539*....
    new_label = np.array(new_label)  # 323539

    return new_feature, new_label, event_num

In [ ]:
new_feature, new_label, event_num = prepare(
    df_drug, feature_list, mechanism, action, drugA, drugB
)
np.random.seed(seed)
np.random.shuffle(new_feature)
np.random.seed(seed)
np.random.shuffle(new_label)
print("dataset len", len(new_feature))

In [ ]:
print(new_feature.shape)

In [ ]:
print(new_label.shape)

In [ ]:
class CNN_DDI(nn.Module):
    def __init__(self):
        super(CNN_DDI, self).__init__()
        self.conv1 = nn.Conv2d(2, 64, (3, 1), padding=(1, 0))
        self.conv2 = nn.Conv2d(64, 128, (3, 1), padding=(1, 0))
        self.conv3_1 = nn.Conv2d(128, 128, (3, 1), padding=(1, 0))
        self.conv3_2 = nn.Conv2d(128, 128, (3, 1), padding=(1, 0))
        self.conv4 = nn.Conv2d(128, 256, (3, 1), padding=(1, 0))
        self.fc1 = nn.Linear(256 * 572 * 4, 256)  # Adjust feature_size based on your input dimensions
        self.fc2 = nn.Linear(256, event_num)  # Assuming 65 DDI types

    def forward(self, x):
        x = x.reshape(-1, 2, 572, 4)
        x = F.leaky_relu(self.conv1(x))
        x = F.leaky_relu(self.conv2(x))
        identity = x
        x = F.leaky_relu(self.conv3_1(x))
        x = self.conv3_2(x)
        x += identity
        x = F.leaky_relu(x)
        x = F.leaky_relu(self.conv4(x))
        x = torch.flatten(x, 1)
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

In [ ]:
class DDIDataset(Dataset):
    def __init__(self, x, y):
        self.len = len(x)
        self.x_data = torch.from_numpy(x)

        self.y_data = torch.from_numpy(y)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [ ]:
class focal_loss(nn.Module):
    def __init__(self, gamma=2):
        super(focal_loss, self).__init__()

        self.gamma = gamma

    def forward(self, preds, labels):
        # assert preds.dim() == 2 and labels.dim()==1
        labels = labels.view(-1, 1).type(torch.int64)  # [B * S, 1]
        preds = preds.view(-1, preds.size(-1))  # [B * S, C]

        preds_logsoft = F.log_softmax(preds, dim=1)  # 先softmax, 然后取log
        preds_softmax = torch.exp(preds_logsoft)  # softmax

        preds_softmax = preds_softmax.gather(1, labels)  # 这部分实现nll_loss ( crossempty = log_softmax + nll )
        preds_logsoft = preds_logsoft.gather(1, labels)

        loss = -torch.mul(torch.pow((1 - preds_softmax), self.gamma),
                          preds_logsoft)  # torch.pow((1-preds_softmax), self.gamma) 为focal loss中 (1-pt)**γ

        loss = loss.mean()

        return loss

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR

train_epochs_loss = []
valid_epochs_loss = []

def train_fn(model, x_train, y_train, x_test, y_test, event_num):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)
    # optimizer = optim.RAdam(model.parameters(), lr=learn_rating, weight_decay=weight_decay_rate)

    # Ranger(RAdam+LookAhead) + CosineAnnealingLR: 50epochs一个半周期
    #optimizer = Ranger(model.parameters(), lr=learn_rating, weight_decay=weight_decay_rate, betas=(0.95, 0.999), eps=1e-6)
    #scheduler = CosineAnnealingLR(optimizer, T_max=epo_num, eta_min=1e-6)


    my_loss = focal_loss()
    model = model.to(device)
    #earlystop = EarlyStopping(patience=patience, delta=delta)

    x_train = np.vstack(
        (
            x_train,
            np.hstack(
                (x_train[:, len(x_train[0]) // 2 :], x_train[:, : len(x_train[0]) // 2])
            ),
        )
    )
    y_train = np.hstack((y_train, y_train))
    np.random.seed(seed)
    np.random.shuffle(x_train)
    np.random.seed(seed)
    np.random.shuffle(y_train)

    len_train = len(y_train)
    len_test = len(y_test)
    print("arg train len", len(y_train))
    print("test len", len(y_test))

    train_dataset = DDIDataset(x_train, np.array(y_train))
    test_dataset = DDIDataset(x_test, np.array(y_test))
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(epo_num):

        running_loss = 0.0

        model.train()
        for batch_idx, data in enumerate(train_loader, 0):
            x, y = data

            # mixup
            lam = np.random.beta(0.5, 0.5)
            index = torch.randperm(x.size()[0])
            inputs = lam * x + (1 - lam) * x[index, :]

            targets_a, targets_b = y, y[index]

            inputs = inputs.to(device)
            targets_a = targets_a.to(device)
            targets_b = targets_b.to(device)

            optimizer.zero_grad()
            # forward + backward+update
            pred = model(inputs.float())

            loss = lam * my_loss(pred, targets_a) + (1 - lam) * my_loss(pred, targets_b)

            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        testing_loss = 0.0
        with torch.no_grad():
            for batch_idx, data in enumerate(test_loader, 0):
                inputs, target = data

                inputs = inputs.to(device)

                target = target.to(device)

                pred = model(inputs.float())

                loss = my_loss(pred, target)
                testing_loss += loss.item()
        # if epoch+1 % 5 == 0:
        print(
            "epoch [%d] loss: %.6f testing_loss: %.6f "
            % (epoch + 1, running_loss / len_train, testing_loss / len_test)
        )
        
        train_epochs_loss.append(running_loss / len_train)
        valid_epochs_loss.append(testing_loss / len_test)
        earlystop(valid_epochs_loss[-1], model, file_path)
        if earlystop.early_stop:
            print("Early stopping\n")
            break


    pre_score = np.zeros((0, event_num), dtype=float)
    model.eval()
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader, 0):
            inputs, _ = data
            inputs = inputs.to(device)
            X = model(inputs.float())
            pre_score = np.vstack((pre_score, F.softmax(X).cpu().numpy()))
    return pre_score

In [ ]:
def cross_val(feature, label, event_num):
    skf = StratifiedKFold(n_splits=5)
    y_true = np.array([])
    y_score = np.zeros((0, event_num), dtype=float)
    y_pred = np.array([])

    for train_index, test_index in skf.split(feature, label):

        model = CNN_DDI()

        X_train, X_test = feature[train_index], feature[test_index]
        y_train, y_test = label[train_index], label[test_index]
        print("train len", len(y_train))
        print("test len", len(y_test))

        pred_score = train_fn(model, X_train, y_train, X_test, y_test, event_num)

        pred_type = np.argmax(pred_score, axis=1)
        y_pred = np.hstack((y_pred, pred_type))
        y_score = np.row_stack((y_score, pred_score))

        y_true = np.hstack((y_true, y_test))

    result_all, result_eve = evaluate(y_pred, y_score, y_true, event_num)
    print("Training finished!")

    return result_all, result_eve

In [ ]:
file_path = "./"
result_all, result_eve = cross_val(new_feature, new_label, event_num)
save_result(file_path, "all", result_all)
save_result(file_path, "each", result_eve)